<a href="https://colab.research.google.com/github/moonlight-t1/42Curcus/blob/master/Machine_Learning/Perfect_Guide/04-2_Human_Activity_Recognition_Using_Smartphones_Data_Set.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 결정 트리 실습 - Human Activity Recognition

- [Human Activity Recognition Using Smartphone Data Set](https://archive.ics.uci.edu/ml/datasets/human+activity+recognition+using+smartphones)
- 30명의 스마트폰 센서를 장착한 뒤 사람의 동작과 관련된 여러 가지 피처를 수집한 데이터이다.
- 수집한 피처 세트를 기반을 결정 트리를 이용해 어떠한 동작인이 예측을 해본다.

In [ ]:
# Colab에 파일 업로드
from google.colab import files

uploaded = files.upload()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# features.txt 파일에는 피처 이름 index와 피처명이 공백으로 분리되어 있다.
# 이를 DataFrame으로 로드
feature_name_df = pd.read_csv('features.txt',sep='\s+',
                        header=None,names=['column_index','column_name'])

# 피처명 index를 제거하고, 피처명만 리스트 객체로 생성한 뒤 샘플로 10개만 추출
feature_name = feature_name_df.iloc[:, 1].values.tolist()
print('전체 피처명에서 10개만 추출:', feature_name[:10])
feature_name_df.head(20)

- 피처명을 보면 인체의 움직임과 관련된 속성의 평균/표준편차가 X, Y, Z축 값으로 돼 있음을 유추할 수 있다.


- 피어명을 가지고 있는 `feature_info.txt` 파일은 중복된 피처명을 가지고 있다.
- 이 중복된 피처명들을 이용해 데이터 파일을 데이터 세트 DataFrame에 로드하면 오류가 발생한다. 
- 따라서 중복된 피처명에 대해서는 원본 피처명에 `_1` 또는 `_2`를 추가로 부여해 변경한 뒤에 이를 이용해서 DataFrame에 로드해야 한다.

In [ ]:
# 중복된 피처명이 얼마나 있는지 확인
feature_dup_df = feature_name_df.groupby('column_name').count()
print(feature_dup_df[feature_dup_df['column_index'] > 1].count())
feature_dup_df[feature_dup_df['column_index'] > 1].head()

- 총 42개의 피처명이 중복돼 있다.
- 이 중복된 피처명에 대해서는 원본 피처명에 _1 또는 _2를 추가로 부여해 새로운 피처명을 가지는 DataFrame을 반환하는 함수인 `get_new_feature_name_df()`를 생성해준다.

In [ ]:
# 중복된 피처명에 대해 새로운 피처명을 생성해주는 함수
def get_new_feature_name_df(old_feature_name_df):
    feature_dup_df = pd.DataFrame(data=old_feature_name_df.groupby('column_name').cumcount(), columns=['dup_cnt'])
    feature_dup_df = feature_dup_df.reset_index()
    new_feature_name_df = pd.merge(old_feature_name_df.reset_index(), feature_dup_df, how='outer')
    new_feature_name_df['column_name'] = new_feature_name_df[['column_name', 'dup_cnt']].apply(lambda x : x[0]+'_'+str(x[1]) 
                                                                                           if x[1] >0 else x[0] ,  axis=1)
    new_feature_name_df = new_feature_name_df.drop(['index'], axis=1)
    return new_feature_name_df

In [ ]:
# 학습용 피처 데이터 세트와 레이블 데이터 세트
# 테스트용 피처 데이터 파일과 레이블 데이터 파일
# 각각 학습/테스트용 DataFrame에 로드해주는 함수
import pandas as pd

def get_human_dataset( ):
    
    # 각 데이터 파일들은 공백으로 분리되어 있으므로 read_csv에서 공백 문자를 sep으로 할당.
    feature_name_df = pd.read_csv('features.txt',sep='\s+',
                        header=None,names=['column_index','column_name'])
    
    # 중복된 feature명을 새롭게 수정하는 get_new_feature_name_df()를 이용하여 새로운 feature명 DataFrame생성. 
    new_feature_name_df = get_new_feature_name_df(feature_name_df)
    
    # DataFrame에 피처명을 컬럼으로 부여하기 위해 리스트 객체로 다시 변환
    feature_name = new_feature_name_df.iloc[:, 1].values.tolist()
    
    # 학습 피처 데이터 셋과 테스트 피처 데이터을 DataFrame으로 로딩. 컬럼명은 feature_name 적용
    X_train = pd.read_csv('X_train.txt',sep='\s+', names=feature_name )
    X_test = pd.read_csv('X_test.txt',sep='\s+', names=feature_name)
    
    # 학습 레이블과 테스트 레이블 데이터을 DataFrame으로 로딩하고 컬럼명은 action으로 부여
    y_train = pd.read_csv('y_train.txt',sep='\s+',header=None,names=['action'])
    y_test = pd.read_csv('y_test.txt',sep='\s+',header=None,names=['action'])
    
    # 로드된 학습/테스트용 DataFrame을 모두 반환 
    return X_train, X_test, y_train, y_test


X_train, X_test, y_train, y_test = get_human_dataset()

- 공백으로 구분되어 있어 csv을 읽을 때 `sep`인자로 공백 문자를 입력해준다.
- 레이블 칼럼은 `action`으로 명명한다.

In [ ]:
# 로드된 학습용 피처 데이터 세트 정보 확인
print('## 학습 피처 데이터셋 info()')
print(X_train.info())

- 학습 데이터 세트는 7352개의 레코드로 561개의 피처를 가지고 있다. 
- 피처가 전부 float 형의 숫자형이므로 별도의 카테고리 인코딩은 수행할 필요가 없다.
- 직접 `X_train.head()`로 간략하게 학습용 피처 데이터 세트를 보면 많은 칼럼의 대부분이 움직임 위치와 관련된 속성임을 알 수 있다.


In [ ]:
print(y_train['action'].value_counts())

- 레이블 값은 1, 2, 3, 4, 5, 6의 6개 값이고 분포도는 특정 값으로 왜곡되지 않고 비교적 고루게 분포되어 있다.

In [ ]:
X_train.isna().sum().sum()

In [ ]:
# 사이킷런의 DecisionTreeClassifier를 이용해 동작 예측 분류를 수행

# 먼저 DecisionTreeClassfier의 하이퍼 파라미터는 모두 디폴트 값으로 설정해 수행하고,
# 이 때의 하이퍼 파라미터 값을 모두 추출해준다.
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# 예제 반복 시 마다 동일한 예측 결과 도출을 위해 random_state 설정
dt_clf = DecisionTreeClassifier(random_state=156)
dt_clf.fit(X_train , y_train)
pred = dt_clf.predict(X_test)
accuracy = accuracy_score(y_test , pred)
print('결정 트리 예측 정확도: {0:.4f}'.format(accuracy))

# DecisionTreeClassifier의 하이퍼 파라미터 추출
print('DecisionTreeClassifier 기본 하이퍼 파라미터:\n', dt_clf.get_params())

- 약 85.48%의 정확도를 나타내고 있다.

In [ ]:
# 결정 트리의 트리 깊이(Tree Depth)가 예측 정확도에 주는 영향을 알아본다.

# GridSeachCV를 이용해 사이컷런 결정 트리의 깊이를 조절할 수 있는 하이퍼 파라미터인 
# max_depth 값을 변화시키면서 예측성능 확인
from sklearn.model_selection import GridSearchCV

params = {
    'max_depth' : [ 6, 8 ,10, 12, 16 ,20, 24]
}

grid_cv = GridSearchCV(dt_clf, param_grid=params, scoring='accuracy', cv=5, verbose=1 )
grid_cv.fit(X_train , y_train)
print('GridSearchCV 최고 평균 정확도 수치:{0:.4f}'.format(grid_cv.best_score_))
print('GridSearchCV 최적 하이퍼 파라미터:', grid_cv.best_params_)

- 결정 트리의 경우 분류를 위해 리프 노드(클래스 결정 노드)가 될 수 있는 적합한 수준이 될 떄까지 지속해서 트리의 분할을 수행한다.
- `max_depth`가 8일 때 5개의 폴드 세티의 최고 평균 정확도 결과가 약 85.26%로 도출됐다.

In [ ]:
# 5개의 CV 세트에서 max_depth 값에 따라 어떻게 예측 성능이 변했는지
# GridSeachCV 객체의 cv_results 속성을 통해 살펴본다.

# GridSearchCV객체의 cv_results_ 속성을 DataFrame으로 생성. 
cv_results_df = pd.DataFrame(grid_cv.cv_results_)

# max_depth 파라미터 값과 그때의 테스트(Evaluation)셋, 학습 데이터 셋의 정확도 수치 추출
# 사이킷런 버전이 업그레이드 되면서 아래의 GridSearchCV 객체의 cv_results_에서 mean_train_score는 더이상 제공되지 않습니다
# cv_results_df[['param_max_depth', 'mean_test_score', 'mean_train_score']]

# max_depth 파라미터 값과 그때의 테스트(Evaluation)셋, 학습 데이터 셋의 정확도 수치 추출
cv_results_df[['param_max_depth', 'mean_test_score']]


# max_dept에 따른 평가 데이터 세트의 평균 정확도 수치(cv_results - mean_test_score)를
# cv_results에서 추출

- GridSearchCV 객체의 cv_results 속성은 CV 세트에 하이퍼 파라미터를 순차적으로 입력했을 떄의 성능 수치를 가지고 있다.


- `mean_test_score`는 5개 CV 세트에서 검증용 데이터 세트의 정확도 평균 수치이다. 
- `mean_test_score`는 `max_depth`가 8일 때 0.852로 정확도가 정점이고, 이를 넘어가면서 정확도가 계속 떨어진다.


- 경정 트리는 더 완벽한 규칙을 학습 데이터 세트에 적용하기 위해 노드를 지속적으로 분할하면서 깊이기 깊어지고 더욱 더 복잡한 모델이 된다.
- 깊어진 트리는 학습 데이터 세트에는 올바른 예측 결과를 가져올지 모르지만, 검증 데이터 세트에서는 오히려 과적합으로 인한 성능 저하를 유발하게 된다.

In [ ]:
# 별도의 테스트 데이터 세트에서 max_depth의 변화에 따른 값을 측정
max_depths = [ 6, 8 ,10, 12, 16 ,20, 24]
# max_depth 값을 변화 시키면서 그때마다 학습과 테스트 셋에서의 예측 성능 측정
for depth in max_depths:
    dt_clf = DecisionTreeClassifier(max_depth=depth, random_state=156)
    dt_clf.fit(X_train , y_train)
    pred = dt_clf.predict(X_test)
    accuracy = accuracy_score(y_test , pred)
    print('max_depth = {0} 정확도: {1:.4f}'.format(depth , accuracy))

- `max_depth`가 8일 경우 87.07%로 가장 높은 정확도를 보여준다. 
- 그리고 `max_depth`가 8을 넘어가면서 정확도가 계속 감소하고 있다.
- `GridSearchCV` 예제와 마찬가지로 깊이가 깊어질수록 테스트 데이터 세트의 정확도는 더 떨어진다. 


- 이 처럼 결정 트리는 깊이가 깊어질수록 과적합의 영향력이 커지므로 하이퍼 파라미터를 이용해 깊이를 제어할 수 있어야 한다.
- 복잡한 모델보다도 트리 깊이를 낮춘 단순한 모델이 더욱 효과적인 결과를 가져올 수 있다.

In [ ]:
# max_depth와 min_samples_split을 같이 변경하면서 정확도 성능을 튜닝
params = {
    'max_depth' : [ 8 , 12, 16 ,20], 
    'min_samples_split' : [16,24],
}

grid_cv = GridSearchCV(dt_clf, param_grid=params, scoring='accuracy', cv=5, verbose=1 )
grid_cv.fit(X_train , y_train)
print('GridSearchCV 최고 평균 정확도 수치: {0:.4f}'.format(grid_cv.best_score_))
print('GridSearchCV 최적 하이퍼 파라미터:', grid_cv.best_params_)


- `max_depth`가 8, `min_samples_split`이 16일 때 가장 최고의 정확도로 약 85.5%를 나타낸다.

In [ ]:
# 별도로 분리된 테스트 데이터 세트에 위에서 구한 하이퍼 파라미터를 적용해본다.
best_df_clf = grid_cv.best_estimator_

pred1 = best_df_clf.predict(X_test)
accuracy = accuracy_score(y_test , pred1)
print('결정 트리 예측 정확도:{0:.4f}'.format(accuracy))

- 앞 예제의 `GridSearchCV` 객체인 `grid_cv` 속성인 `best_estimator_`는 최적 하이퍼 파라미터인 `max_depth` 8, `min_samples_split` 16으로 학습이 완료된 `Estimator` 객체이다.
- `max_depth` 8, `min_samples_split` 16 일 때 테스트 데이터 세트의 예측 정확도는 약 87.17% 이다.

In [ ]:
# 결정 트리에서 각 피처의 중요도를 feature_importances_ 속성을 이용해 알아본다

import seaborn as sns

ftr_importances_values = best_df_clf.feature_importances_

# Top 중요도로 정렬을 쉽게 하고, 시본(Seaborn)의 막대그래프로 쉽게 표현하기 위해 Series변환
ftr_importances = pd.Series(ftr_importances_values, index=X_train.columns  )

# 중요도값 순으로 Series를 정렬
ftr_top20 = ftr_importances.sort_values(ascending=False)[:20]
plt.figure(figsize=(8,6))
plt.title('Feature importances Top 20')
sns.barplot(x=ftr_top20 , y = ftr_top20.index)
plt.show()

- 막대 그래프를 확인해 보면 이 중 가장 높은 중요도를 가진 Top 5의 피처들이 매우 중요하게 규칙 생성에 영향을 미치고 있다.